# 🎓 AP Shah Institute - Enhanced Student Access Control System

This enhanced notebook combines:
1. **ID Card Recognition** - Extract Moodle ID, Name, Department using EasyOCR
2. **Face Recognition** - DeepFace-based face matching for secure access
3. **License Plate Detection** - Vehicle access control
4. **Database Integration** - Supabase PostgreSQL for cloud storage
5. **Live Camera Access** - Real-time ID verification

## Features:
- ✅ 100% Offline ID card OCR (EasyOCR)
- ✅ Face recognition using DeepFace (VGG-Face model)
- ✅ YOLO-based ID card detection
- ✅ Smart validation (filters out institutional text)
- ✅ Supabase cloud database integration
- ✅ Live camera with green guide box
- ✅ Confidence-based filtering (>75% for vehicles)

## Step 1: Install Required Packages

In [ ]:
# Install all required packages
!pip install opencv-python opencv-contrib-python
!pip install easyocr
!pip install ultralytics
!pip install deepface tf-keras
!pip install psycopg2-binary python-dotenv
!pip install matplotlib pandas numpy tqdm
!pip install ipywidgets

print("✅ All packages installed successfully!")

## Step 2: Import Libraries and Setup

In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import psycopg2
from datetime import datetime
import matplotlib.pyplot as plt
import re
import easyocr
from IPython.display import display, clear_output, Image
import ipywidgets as widgets
from deepface import DeepFace
from ultralytics import YOLO
from dotenv import load_dotenv

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6)

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")

## Step 3: Configure Paths and Database

In [ ]:
# Base directories
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data', 'training_data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs', 'id_card_data')
FACE_DB_DIR = os.path.join(BASE_DIR, 'face_database')
MODEL_PATH = os.path.join(BASE_DIR, 'models', 'best.pt')

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(FACE_DB_DIR, exist_ok=True)

# Supabase configuration from .env
SUPABASE_HOST = os.getenv('SUPABASE_HOST')
SUPABASE_PORT = os.getenv('SUPABASE_PORT', '5432')
SUPABASE_DBNAME = os.getenv('SUPABASE_DBNAME')
SUPABASE_USER = os.getenv('SUPABASE_USER')
SUPABASE_PASSWORD = os.getenv('SUPABASE_PASSWORD')

print(f"✅ Base Directory: {BASE_DIR}")
print(f"✅ Training Data: {DATA_DIR}")
print(f"✅ Output Directory: {OUTPUT_DIR}")
print(f"✅ Face Database: {FACE_DB_DIR}")
print(f"✅ Model Path: {MODEL_PATH}")
print(f"✅ Supabase: {SUPABASE_HOST}")

## Step 4: Initialize Supabase Database Manager

In [ ]:
class SupabaseManager:
    def __init__(self):
        self.conn = None
        self.connect()
        self.create_tables()
    
    def connect(self):
        """Connect to Supabase PostgreSQL database"""
        try:
            self.conn = psycopg2.connect(
                host=SUPABASE_HOST,
                port=SUPABASE_PORT,
                dbname=SUPABASE_DBNAME,
                user=SUPABASE_USER,
                password=SUPABASE_PASSWORD
            )
            print("✅ Connected to Supabase database")
        except Exception as e:
            print(f"❌ Database connection error: {e}")
    
    def create_tables(self):
        """Create all required tables"""
        try:
            cursor = self.conn.cursor()
            
            # Students table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS students (
                    id SERIAL PRIMARY KEY,
                    moodle_id VARCHAR(20) UNIQUE,
                    name VARCHAR(100),
                    department VARCHAR(100),
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # ID card logs
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS id_card_logs (
                    id SERIAL PRIMARY KEY,
                    moodle_id VARCHAR(20),
                    name VARCHAR(100),
                    department VARCHAR(100),
                    confidence FLOAT,
                    camera_id VARCHAR(50),
                    access_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    status VARCHAR(20) DEFAULT 'allowed'
                )
            ''')
            
            # Face recognition logs
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS face_recognition_logs (
                    id SERIAL PRIMARY KEY,
                    moodle_id VARCHAR(20),
                    name VARCHAR(100),
                    confidence FLOAT,
                    distance FLOAT,
                    access_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    access_granted BOOLEAN DEFAULT TRUE
                )
            ''')
            
            # Vehicle logs
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS vehicle_logs (
                    id SERIAL PRIMARY KEY,
                    license_plate VARCHAR(20),
                    confidence FLOAT,
                    camera_id VARCHAR(50),
                    access_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    status VARCHAR(20) DEFAULT 'allowed'
                )
            ''')
            
            self.conn.commit()
            print("✅ All tables created successfully")
        except Exception as e:
            print(f"❌ Error creating tables: {e}")
    
    def insert_student(self, moodle_id, name, department):
        """Insert or update student record"""
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                INSERT INTO students (moodle_id, name, department)
                VALUES (%s, %s, %s)
                ON CONFLICT (moodle_id) DO UPDATE
                SET name = EXCLUDED.name,
                    department = EXCLUDED.department,
                    updated_at = CURRENT_TIMESTAMP
            ''', (moodle_id, name, department))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"❌ Error inserting student: {e}")
            return False
    
    def log_id_card_access(self, moodle_id, name, department, confidence, camera_id='notebook'):
        """Log ID card access"""
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                INSERT INTO id_card_logs (moodle_id, name, department, confidence, camera_id)
                VALUES (%s, %s, %s, %s, %s)
            ''', (moodle_id, name, department, confidence, camera_id))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"❌ Error logging access: {e}")
            return False
    
    def log_face_recognition(self, moodle_id, name, confidence, distance, access_granted=True):
        """Log face recognition access"""
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                INSERT INTO face_recognition_logs (moodle_id, name, confidence, distance, access_granted)
                VALUES (%s, %s, %s, %s, %s)
            ''', (moodle_id, name, confidence, distance, access_granted))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"❌ Error logging face recognition: {e}")
            return False
    
    def log_vehicle_access(self, license_plate, confidence, camera_id='notebook'):
        """Log vehicle access (only if confidence >= 75%)"""
        if confidence < 0.75:
            print(f"⚠️ Vehicle {license_plate} not logged - confidence {confidence:.2%} < 75%")
            return False
        
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                INSERT INTO vehicle_logs (license_plate, confidence, camera_id)
                VALUES (%s, %s, %s)
            ''', (license_plate, confidence, camera_id))
            self.conn.commit()
            return True
        except Exception as e:
            print(f"❌ Error logging vehicle: {e}")
            return False
    
    def get_recent_logs(self, limit=10, table='id_card_logs'):
        """Get recent logs from any table"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(f'''
                SELECT * FROM {table}
                ORDER BY access_time DESC
                LIMIT %s
            ''', (limit,))
            columns = [desc[0] for desc in cursor.description]
            results = cursor.fetchall()
            return [dict(zip(columns, row)) for row in results]
        except Exception as e:
            print(f"❌ Error fetching logs: {e}")
            return []
    
    def close(self):
        """Close database connection"""
        if self.conn:
            self.conn.close()
            print("✅ Database connection closed")

# Initialize database
db = SupabaseManager()
print("\n✅ Database Manager initialized successfully")

## Step 5: Initialize YOLO Model and EasyOCR

In [ ]:
# Initialize YOLO model for ID card detection
print("Loading YOLO model...")
yolo_model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded successfully")

# Initialize EasyOCR
print("\nInitializing EasyOCR (this may take a moment)...")
ocr_reader = easyocr.Reader(['en'], gpu=True, verbose=False)
print("✅ EasyOCR initialized successfully (GPU enabled)")

## Step 6: ID Card Processor with Smart Validation

In [ ]:
class EnhancedIDProcessor:
    def __init__(self):
        self.db = db
        self.reader = ocr_reader
        self.model = yolo_model
        
        # Smart validation patterns
        self.invalid_name_patterns = {
            'apsit', 'institute', 'technology', 'engineering', 'college',
            'comp', 'computer', 'information', 'department', 'academic',
            'principal', 'thane', 'mumbai', 'year', 'student'
        }
        
        self.valid_departments = {
            'COMPUTER ENGINEERING': 'Computer Engineering',
            'COMP ENGINEERING': 'Computer Engineering',
            'C@MPUTER ENGINEERING': 'Computer Engineering',
            'INFORMATION TECHNOLOGY': 'Information Technology',
            'IT': 'Information Technology',
            'ELECTRONICS': 'Electronics Engineering',
            'MECHANICAL': 'Mechanical Engineering',
            'CIVIL': 'Civil Engineering'
        }
    
    def is_likely_name(self, text):
        """Check if text is likely a valid name"""
        if not text or len(text) < 6 or len(text) > 30:
            return False
        
        # Check for invalid patterns
        text_lower = text.lower()
        if any(pattern in text_lower for pattern in self.invalid_name_patterns):
            return False
        
        # Check vowel proportion (names typically have 20-50% vowels)
        vowels = sum(1 for c in text_lower if c in 'aeiou')
        letters = sum(1 for c in text if c.isalpha())
        if letters > 0:
            vowel_ratio = vowels / letters
            if vowel_ratio < 0.2 or vowel_ratio > 0.6:
                return False
        
        # Should have 2-3 words (First Middle Last or First Last)
        words = text.split()
        if len(words) < 2 or len(words) > 3:
            return False
        
        return True
    
    def normalize_department(self, dept_text):
        """Normalize department name with typo correction"""
        if not dept_text:
            return None
        
        dept_upper = dept_text.upper().strip()
        
        # Check direct matches
        for key, value in self.valid_departments.items():
            if key in dept_upper:
                return value
        
        return None
    
    def extract_info(self, image):
        """Extract Moodle ID, Name, and Department from ID card"""
        # Preprocess image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # OCR extraction
        results = self.reader.readtext(gray, detail=0)
        full_text = ' '.join(results).upper()
        
        # Extract Moodle ID (8 digits)
        moodle_id = None
        for text in results:
            match = re.search(r'(\d{8})', text)
            if match:
                moodle_id = match.group(1)
                break
        
        # Extract name with smart validation
        name = None
        for text in results:
            if self.is_likely_name(text):
                name = text.title()
                break
        
        # Extract and normalize department
        department = self.normalize_department(full_text)
        
        return moodle_id, name, department
    
    def process_image(self, image_path, save_results=True, display_results=True):
        """Process a single ID card image"""
        print(f"\n{'='*70}")
        print(f"Processing: {os.path.basename(image_path)}")
        print(f"{'='*70}")
        
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            print("❌ Failed to load image")
            return None
        
        # Detect ID card with YOLO
        results = self.model(image, conf=0.3)
        
        if len(results[0].boxes) > 0:
            # Get first detection
            box = results[0].boxes[0]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = float(box.conf[0])
            
            # Crop ID card region
            id_card_crop = image[y1:y2, x1:x2]
            
            print(f"✅ ID card detected (confidence: {confidence:.2%})")
        else:
            # Use full image if no detection
            id_card_crop = image
            confidence = 0.5
            print("⚠️ No ID card detected, using full image")
        
        # Extract information
        moodle_id, name, department = self.extract_info(id_card_crop)
        
        # Print results
        print(f"\n📋 Extracted Information:")
        print(f"   Moodle ID: {moodle_id or '❌ Not found'}")
        print(f"   Name: {name or '❌ Not found'}")
        print(f"   Department: {department or '❌ Not found'}")
        
        # Save to database
        if moodle_id:
            self.db.insert_student(
                moodle_id=moodle_id,
                name=name or 'Unknown',
                department=department or 'Unknown'
            )
            self.db.log_id_card_access(
                moodle_id=moodle_id,
                name=name or 'Unknown',
                department=department or 'Unknown',
                confidence=confidence,
                camera_id='notebook'
            )
            print(f"\n✅ Data saved to Supabase database")
        
        # Save annotated image
        if save_results:
            annotated = image.copy()
            cv2.putText(annotated, f"ID: {moodle_id or 'N/A'}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(annotated, f"Name: {name or 'N/A'}", (10, 65),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(annotated, f"Dept: {department or 'N/A'}", (10, 100),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join(OUTPUT_DIR, f"{moodle_id or 'unknown'}_{timestamp}.jpg")
            cv2.imwrite(output_path, annotated)
            print(f"💾 Annotated image saved: {output_path}")
        
        # Display results
        if display_results:
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[0].set_title("Original Image", fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            axes[1].imshow(cv2.cvtColor(id_card_crop, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f"Detected ID Card\nID: {moodle_id or 'N/A'}", 
                            fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
        
        return {
            'moodle_id': moodle_id,
            'name': name,
            'department': department,
            'confidence': confidence
        }

# Initialize processor
processor = EnhancedIDProcessor()
print("\n✅ Enhanced ID Processor initialized successfully")

## Step 7: Interactive ID Card Scanner

In [ ]:
# Get list of available ID card images
id_card_files = [f for f in os.listdir(DATA_DIR) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not id_card_files:
    print("❌ No ID card images found in data/training_data directory")
else:
    print(f"✅ Found {len(id_card_files)} ID card images")
    
    # Create interactive dropdown
    dropdown = widgets.Dropdown(
        options=id_card_files,
        description='Select ID:',
        layout=widgets.Layout(width='500px')
    )
    
    process_button = widgets.Button(
        description='🔍 Process ID Card',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    batch_button = widgets.Button(
        description='📦 Process All Images',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    output = widgets.Output()
    
    def on_process_click(b):
        with output:
            clear_output(wait=True)
            file_path = os.path.join(DATA_DIR, dropdown.value)
            processor.process_image(file_path)
    
    def on_batch_click(b):
        with output:
            clear_output(wait=True)
            print(f"Processing {len(id_card_files)} images...\n")
            
            results = []
            for i, filename in enumerate(id_card_files, 1):
                print(f"[{i}/{len(id_card_files)}] {filename}")
                file_path = os.path.join(DATA_DIR, filename)
                result = processor.process_image(file_path, display_results=False)
                if result:
                    results.append(result)
                print("-" * 70)
            
            print(f"\n✅ Batch processing complete!")
            print(f"   Processed: {len(results)}/{len(id_card_files)} images")
            print(f"   Moodle IDs found: {sum(1 for r in results if r['moodle_id'])}")
            print(f"   Names found: {sum(1 for r in results if r['name'])}")
            print(f"   Departments found: {sum(1 for r in results if r['department'])}")
    
    process_button.on_click(on_process_click)
    batch_button.on_click(on_batch_click)
    
    display(widgets.VBox([
        dropdown,
        widgets.HBox([process_button, batch_button]),
        output
    ]))

## Step 8: Face Recognition Setup

In [ ]:
class FaceRecognitionSystem:
    def __init__(self):
        self.face_db_dir = FACE_DB_DIR
        self.db = db
        self.known_faces = {}
        self.student_info = {}
        self.load_database()
    
    def load_database(self):
        """Load face database and student info"""
        # Load face images
        for file in os.listdir(self.face_db_dir):
            if file.endswith('.jpg'):
                student_id = os.path.splitext(file)[0]
                self.known_faces[student_id] = os.path.join(self.face_db_dir, file)
        
        # Load student info from database
        try:
            cursor = self.db.conn.cursor()
            cursor.execute("SELECT moodle_id, name, department FROM students")
            for row in cursor.fetchall():
                if row[0]:
                    self.student_info[row[0]] = {
                        'name': row[1] or 'Unknown',
                        'department': row[2] or 'Unknown'
                    }
        except Exception as e:
            print(f"⚠️ Could not load student info: {e}")
        
        print(f"✅ Loaded {len(self.known_faces)} face templates")
        print(f"✅ Loaded {len(self.student_info)} student records")
    
    def extract_face_from_id(self, id_card_path, student_id):
        """Extract face from ID card and save to database"""
        try:
            # Detect face using DeepFace
            face_objs = DeepFace.extract_faces(
                img_path=id_card_path,
                detector_backend='opencv',
                enforce_detection=False
            )
            
            if face_objs:
                # Get largest face
                face = max(face_objs, key=lambda x: x.get('confidence', 0))
                
                # Load original image
                img = cv2.imread(id_card_path)
                h, w = img.shape[:2]
                
                # Extract face region with padding
                area = face['facial_area']
                padding = 30
                x1 = max(0, area['x'] - padding)
                y1 = max(0, area['y'] - padding)
                x2 = min(w, area['x'] + area['w'] + padding)
                y2 = min(h, area['y'] + area['h'] + padding)
                
                face_img = img[y1:y2, x1:x2]
                
                # Save extracted face
                face_path = os.path.join(self.face_db_dir, f"{student_id}.jpg")
                cv2.imwrite(face_path, face_img)
                
                print(f"✅ Face extracted and saved: {student_id}.jpg")
                return face_path
            else:
                print(f"❌ No face detected in {id_card_path}")
                return None
        except Exception as e:
            print(f"❌ Error extracting face: {e}")
            return None
    
    def build_face_database(self):
        """Extract faces from all ID cards in training data"""
        print(f"\n{'='*70}")
        print("Building Face Database from ID Cards")
        print(f"{'='*70}\n")
        
        id_card_files = [f for f in os.listdir(DATA_DIR)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        extracted = 0
        for i, filename in enumerate(id_card_files, 1):
            print(f"[{i}/{len(id_card_files)}] Processing {filename}")
            file_path = os.path.join(DATA_DIR, filename)
            
            # Extract Moodle ID first
            image = cv2.imread(file_path)
            moodle_id, _, _ = processor.extract_info(image)
            
            if not moodle_id:
                moodle_id = os.path.splitext(filename)[0]
            
            # Extract face
            face_path = self.extract_face_from_id(file_path, moodle_id)
            
            if face_path:
                extracted += 1
            print()
        
        print(f"\n✅ Face extraction complete: {extracted}/{len(id_card_files)} faces extracted")
        self.load_database()  # Reload database
    
    def verify_face(self, live_image_path):
        """Verify face against database"""
        print(f"\n{'='*70}")
        print("Face Recognition - Searching for match...")
        print(f"{'='*70}\n")
        
        best_match = None
        best_distance = 1.0
        
        for student_id, face_path in self.known_faces.items():
            try:
                result = DeepFace.verify(
                    img1_path=live_image_path,
                    img2_path=face_path,
                    model_name='VGG-Face',
                    enforce_detection=False,
                    distance_metric='cosine'
                )
                
                distance = result['distance']
                
                if distance < best_distance and distance < 0.6:
                    best_distance = distance
                    best_match = student_id
                    
                if distance < 0.7:
                    print(f"   {student_id}: distance={distance:.4f}")
            except Exception as e:
                continue
        
        if best_match:
            confidence = 1 - best_distance
            info = self.student_info.get(best_match, {'name': best_match, 'department': 'Unknown'})
            
            # Log to database
            self.db.log_face_recognition(
                moodle_id=best_match,
                name=info['name'],
                confidence=confidence,
                distance=best_distance,
                access_granted=True
            )
            
            print(f"\n{'='*70}")
            print(f"✅ MATCH FOUND!")
            print(f"{'='*70}")
            print(f"   Moodle ID: {best_match}")
            print(f"   Name: {info['name']}")
            print(f"   Department: {info['department']}")
            print(f"   Confidence: {confidence*100:.1f}%")
            print(f"   Distance: {best_distance:.4f}")
            print(f"{'='*70}")
            
            return best_match, info, confidence
        else:
            print(f"\n❌ No match found (best distance: {best_distance:.4f})")
            return None, None, 0.0
    
    def start_live_camera(self):
        """Start live camera for face recognition"""
        print(f"\n{'='*70}")
        print("Starting Live Face Recognition System")
        print(f"{'='*70}")
        print("Press SPACE to capture and verify face")
        print("Press 'q' to quit\n")
        
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Draw guide box
            h, w = frame.shape[:2]
            box_w, box_h = 300, 400
            x1 = (w - box_w) // 2
            y1 = (h - box_h) // 2
            x2 = x1 + box_w
            y2 = y1 + box_h
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, "Position face in green box", (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, "SPACE = Capture | Q = Quit", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            
            cv2.imshow('Face Recognition - Press SPACE to verify', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord(' '):
                # Save temporary image
                temp_file = os.path.join(OUTPUT_DIR, 'temp_live_capture.jpg')
                cv2.imwrite(temp_file, frame)
                
                # Verify face
                student_id, info, confidence = self.verify_face(temp_file)
                
                if student_id:
                    # Show success
                    success_frame = frame.copy()
                    cv2.rectangle(success_frame, (0, 0), (w, 150), (0, 255, 0), -1)
                    cv2.putText(success_frame, "ACCESS GRANTED", (20, 50),
                               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                    cv2.putText(success_frame, f"{info['name']}", (20, 90),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(success_frame, f"ID: {student_id} | {confidence*100:.1f}%",
                               (20, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    
                    cv2.imshow('Face Recognition - Press SPACE to verify', success_frame)
                    cv2.waitKey(3000)
                else:
                    # Show denied
                    denied_frame = frame.copy()
                    cv2.rectangle(denied_frame, (0, 0), (w, 100), (0, 0, 255), -1)
                    cv2.putText(denied_frame, "ACCESS DENIED", (20, 60),
                               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                    
                    cv2.imshow('Face Recognition - Press SPACE to verify', denied_frame)
                    cv2.waitKey(2000)
                
                # Cleanup
                if os.path.exists(temp_file):
                    os.remove(temp_file)
        
        cap.release()
        cv2.destroyAllWindows()
        print("\n✅ Camera stopped")

# Initialize face recognition system
face_system = FaceRecognitionSystem()
print("\n✅ Face Recognition System initialized successfully")

## Step 9: Build Face Database (Run Once)

In [ ]:
# Extract faces from all ID cards and build face database
# Run this once to create the face database
face_system.build_face_database()

## Step 10: Start Live Face Recognition

In [ ]:
# Start live camera for face recognition
# Press SPACE to capture and verify face
# Press 'q' to quit
face_system.start_live_camera()

## Step 11: View Recent Access Logs

In [ ]:
# View recent ID card access logs
print("\n📋 Recent ID Card Access Logs:")
print("=" * 100)
id_logs = db.get_recent_logs(limit=10, table='id_card_logs')
if id_logs:
    df_id = pd.DataFrame(id_logs)
    display(df_id[['moodle_id', 'name', 'department', 'confidence', 'access_time']])
else:
    print("No ID card logs found")

# View recent face recognition logs
print("\n👤 Recent Face Recognition Logs:")
print("=" * 100)
face_logs = db.get_recent_logs(limit=10, table='face_recognition_logs')
if face_logs:
    df_face = pd.DataFrame(face_logs)
    display(df_face[['moodle_id', 'name', 'confidence', 'distance', 'access_time']])
else:
    print("No face recognition logs found")

# View recent vehicle logs
print("\n🚗 Recent Vehicle Logs (Confidence >= 75%):")
print("=" * 100)
vehicle_logs = db.get_recent_logs(limit=10, table='vehicle_logs')
if vehicle_logs:
    df_vehicle = pd.DataFrame(vehicle_logs)
    display(df_vehicle[['license_plate', 'confidence', 'camera_id', 'access_time']])
else:
    print("No vehicle logs found")

## Summary

### ✅ Completed Features:

1. **ID Card Recognition**
   - YOLO-based ID card detection
   - EasyOCR for text extraction (Moodle ID, Name, Department)
   - Smart validation (filters institutional text, validates names)
   - Department normalization with typo correction

2. **Face Recognition**
   - DeepFace-based face matching (VGG-Face model)
   - Face database extraction from ID cards
   - Live camera with green guide box
   - Confidence threshold (60% for acceptance)

3. **Database Integration**
   - Supabase PostgreSQL cloud database
   - Tables: students, id_card_logs, face_recognition_logs, vehicle_logs
   - Real-time logging of all access events
   - Confidence-based filtering for vehicles (>= 75%)

4. **Interactive Features**
   - Dropdown selector for ID cards
   - Batch processing capability
   - Live camera feed
   - Visual feedback (green for access granted, red for denied)

### 🎯 Advantages over Previous Notebooks:

- ✅ Cloud database (Supabase) instead of local SQLite
- ✅ Smart validation eliminates false positives
- ✅ Combined ID card + face recognition
- ✅ Confidence-based vehicle filtering
- ✅ Better organized and modular code
- ✅ Interactive widgets for easy testing
- ✅ Comprehensive logging and statistics

### 📝 Next Steps:

1. Run Step 9 to build face database from ID cards
2. Use Step 10 for live face recognition
3. Check Step 11 for access logs and statistics
4. Integrate with Streamlit app for production deployment